# 1. Load library and data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import time
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2017-12-06 19:55:53


In [2]:
input_dir = '../input/'
# input_dir = '/home/inputs/' # for google cloud

for x in glob.glob(input_dir + "*"):
    print(x)

../input\members.csv
../input\sample_submission.csv
../input\songs.csv
../input\song_extra_info.csv
../input\test.csv
../input\train.csv


In [3]:
df_train = pd.read_csv(input_dir + "train.csv")
df_test  = pd.read_csv(input_dir + 'test.csv')
df_songs = pd.read_csv(input_dir + 'songs.csv')
df_song_extra = pd.read_csv(input_dir + "song_extra_info.csv")
df_members = pd.read_csv(input_dir + "members.csv", parse_dates=["registration_init_time","expiration_date"])

# 2. Feature Engineering 

In [4]:
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2017-12-06 19:56:17


## 2.1 Members Dataset
- rename column 'bd' -> 'age' 
- add feature: validate days
- registration_init_time -> registration_year + registration_month + registration_day (2011-08-20 -> 2011 08 20)
- expiration_date -> expiration_year + expiration_month + expiration_day (2011-08-20 -> 2011 08 20)

In [5]:
# Members
df_members.rename(columns={'bd': 'age'}, inplace=True)
df_members.loc[df_members['age'] < 5, 'age'] = 0
df_members.loc[df_members['age'] >= 80, 'age'] = 0
df_members['weird_age'] = 0
df_members.loc[df_members['age'] == 0, 'weird_age'] = 1
df_members['validate_days'] = (df_members['expiration_date'] - df_members['registration_init_time']).dt.days.astype(int)

df_members['registration_year'] = df_members['registration_init_time'].dt.year.astype(int)
df_members['registration_month'] = df_members['registration_init_time'].dt.month.astype(int)
df_members['registration_day'] = df_members['registration_init_time'].dt.day.astype(int)

df_members['expiration_year'] = df_members['expiration_date'].dt.year.astype(int)
df_members['expiration_month'] = df_members['expiration_date'].dt.month.astype(int)
df_members['expiration_day'] = df_members['expiration_date'].dt.day.astype(int)

df_members.drop(['registration_init_time', 'expiration_date'], axis=1, inplace=True)

In [6]:
df_members.head()

,msno,city,age,gender,registered_via,weird_age,validate_days,registration_year,registration_month,registration_day,expiration_year,expiration_month,expiration_day
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,1,2223,2011,8,20,2017,9,20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,1,725,2015,6,28,2017,6,22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,1,457,2016,4,11,2017,7,12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,1,1,2015,9,6,2015,9,7
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,1,138,2017,1,26,2017,6,13


### 2.2 Song related
- isrc -> song_year (TWUM7 12 00043 -> 2012, QMZSY 16 00015 -> 2016, TWA53 08 87303 -> 2008)
- drop column song_name
- preprocessing song length to unit second

In [7]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

df_song_extra['song_year'] = df_song_extra['isrc'].apply(isrc_to_year)
df_song_extra.drop(['isrc', 'name'], axis = 1, inplace = True)
# 1000 <=> 1s
df_songs['song_length'] /= 1000.0
df_songs

## 2.3 Merge all dataset based on song_id and msno(user_id)

In [8]:
# left join train and song based on song_id
# left join train and members based on msno(user id)
# left join train and song_extra based on song_id
df_train = df_train.merge(df_songs, how="left", on="song_id")
df_train = df_train.merge(df_members, how="left", on="msno")
df_train = df_train.merge(df_song_extra, how='left', on='song_id')

df_test  = df_test.merge(df_songs, how="left", on="song_id")
df_test  = df_test.merge(df_members, how="left", on="msno")
df_test = df_test.merge(df_song_extra, how='left', on='song_id')

## Adjust the data type

In [10]:
for col in ['registered_via', 'city', 'language']:
    df_train[col] = df_train[col].astype(str)
    df_test[col] = df_test[col].astype(str)

## 2.4 Fill in missing value
- categorical feature -> 'Unknown'
- continuous feaure -> mean or mode

In [11]:
# find columns with missing value
shared_columns = set(df_train.columns).intersection(df_test.columns)
for col in shared_columns:
    if df_train[col].isnull().any() or df_test[col].isnull().any():
        print('{:10} {}'.format(str(df_train[col].dtype), col))

float64    song_year
object     source_system_tab
object     composer
object     gender
object     artist_name
object     genre_ids
object     source_screen_name
object     source_type
object     lyricist
float64    song_length


In [13]:
# fill categorical columns with tag: 'Unknown'
UNKNOWN = 'Unknown'
col_fill_with_unknown = [
    'source_system_tab', 'source_screen_name', 'source_type', 
    'gender', 
    'genre_ids', 'artist_name', 'composer', 'lyricist', 'language'
]
for col in col_fill_with_unknown:
    df_train[col].fillna(value=UNKNOWN, inplace=True)
    df_test[col].fillna(value=UNKNOWN, inplace=True)
    
# fill in song length with 230 (approximately mean and mode)
df_train['song_length'].fillna(value=230,inplace=True)
df_test['song_length'].fillna(value=230,inplace=True)

fill_in_value = df_train['song_year'].median()
df_train['song_year'].fillna(value=fill_in_value, inplace=True)
df_test['song_year'].fillna(value=fill_in_value, inplace=True)

In [14]:
# check whether all filled
shared_columns = set(df_train.columns).intersection(df_test.columns)
for col in shared_columns:
    if df_train[col].isnull().any() or df_test[col].isnull().any():
        print('{:10} {}'.format(str(df_train[col].dtype), col))

## 2.5 More Feature Engineering
- count features
- binary features

In [15]:
# song_length, clean data
df_train.loc[df_train['song_length'] > 1800, 'song_length'] = 1800
df_test.loc[df_train['song_length'] > 1800, 'song_length'] = 1800

def _count(x):
    if x == UNKNOWN:
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';', '、', ','])) + 1

# the number of genre ids
df_train['genre_count'] = df_train['genre_ids'].apply(_count).astype(int)
df_test['genre_count'] = df_test['genre_ids'].apply(_count).astype(int)

# the number of lyricists
df_train['lyricist_count'] = df_train['lyricist'].apply(_count).astype(int)
df_test['lyricist_count'] = df_test['lyricist'].apply(_count).astype(int)

# the number of composers
df_train['composer_count'] = df_train['composer'].apply(_count).astype(int)
df_test['composer_count'] = df_test['composer'].apply(_count).astype(int)

# the number of artists
def artist_count(x):
    if x == UNKNOWN:
        return 0
    else:
        return x.count('and') + x.count(',') + x.count('feat') + x.count('&') + 1
df_train['artist_count'] = df_train['artist_name'].apply(artist_count).astype(np.int8)
df_test['artist_count'] = df_test['artist_name'].apply(artist_count).astype(np.int8)

# if the artist name includes 'is_featured'
def is_featured(x):
    if 'feat' in str(x) :
        return 1
    return 0
df_train['is_featured'] = df_train['artist_name'].apply(is_featured).astype(np.int8)
df_test['is_featured'] = df_test['artist_name'].apply(is_featured).astype(np.int8)

# if artist is same as composer
df_train['artist_composer'] = (df_train['artist_name'] == df_train['composer']).astype(np.int8)
df_test['artist_composer'] = (df_test['artist_name'] == df_test['composer']).astype(np.int8)

# if artist, lyricist and composer are all three same
df_train['artist_composer_lyricist'] = ((df_train['artist_name'] == df_train['composer']) 
                                        & (df_train['artist_name'] == df_train['lyricist']) 
                                        & (df_train['composer'] == df_train['lyricist'])).astype(np.int8)
df_test['artist_composer_lyricist'] = ((df_test['artist_name'] == df_test['composer']) 
                                       & (df_test['artist_name'] == df_test['lyricist']) 
                                       & (df_test['composer'] == df_test['lyricist'])).astype(np.int8)

# if song language is 17 or 45. 
def song_lang_boolean(x):
    if '17.0' in str(x) or '45.0' in str(x):
        return 1
    return 0
df_train['song_lang_boolean'] = df_train['language'].apply(song_lang_boolean).astype(np.int8)
df_test['song_lang_boolean'] = df_test['language'].apply(song_lang_boolean).astype(np.int8)

# if the song's length is shorter than mean
_mean_song_length = np.mean(df_train['song_length'])
def smaller_song(x):
    if x < _mean_song_length:
        return 1
    return 0
df_train['smaller_song'] = df_train['song_length'].apply(smaller_song).astype(np.int8)
df_test['smaller_song'] = df_test['song_length'].apply(smaller_song).astype(np.int8)

## 2.6 Historical Statistical Data

In [16]:
# number of times a song has been played before
_dict_count_song_played_train = {k: v for k, v in df_train['song_id'].value_counts().iteritems()}
_dict_count_song_played_test = {k: v for k, v in df_test['song_id'].value_counts().iteritems()}
def count_song_played(x):
    try:
        return _dict_count_song_played_train[x]
    except KeyError:
        try:
            return _dict_count_song_played_test[x]
        except KeyError:
            return 0
df_train['count_song_played'] = df_train['song_id'].apply(count_song_played).astype(np.int64)
df_test['count_song_played'] = df_test['song_id'].apply(count_song_played).astype(np.int64)
del _dict_count_song_played_test
del _dict_count_song_played_train

# number of times the artist has been played
_dict_count_artist_played_train = {k: v for k, v in df_train['artist_name'].value_counts().iteritems()}
_dict_count_artist_played_test = {k: v for k, v in df_test['artist_name'].value_counts().iteritems()}
def count_artist_played(x):
    try:
        return _dict_count_artist_played_train[x]
    except KeyError:
        try:
            return _dict_count_artist_played_test[x]
        except KeyError:
            return 0
df_train['count_artist_played'] = df_train['artist_name'].apply(count_artist_played).astype(np.int64)
df_test['count_artist_played'] = df_test['artist_name'].apply(count_artist_played).astype(np.int64)
del _dict_count_artist_played_train
del _dict_count_artist_played_test

# 3. Preparing Data For TensorFlow

In [17]:
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2017-12-06 19:59:13


In [18]:
df_train.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'age', 'gender',
       'registered_via', 'weird_age', 'validate_days', 'registration_year',
       'registration_month', 'registration_day', 'expiration_year',
       'expiration_month', 'expiration_day', 'song_year', 'genre_count',
       'lyricist_count', 'composer_count', 'is_featured', 'artist_count',
       'artist_composer', 'artist_composer_lyricist', 'song_lang_boolean',
       'smaller_song', 'count_song_played', 'count_artist_played'],
      dtype='object')

In [78]:
columns = dict()

## 3.1 Source Related Feature

In [79]:
# Columns with a small set of categories: transform them using categorical_column_with_vocabulary_list
_source_cols = ['source_screen_name', 'source_type', 'source_system_tab']
for col in _source_cols:
    col_set = set(df_train[col].unique()).union(set(df_test[col].unique()))
    columns[col] = tf.feature_column.categorical_column_with_vocabulary_list(key=col, vocabulary_list=col_set)

## 3.2 Members Related Feature

In [80]:
# Columns with a small set of categories: transform them using categorical_column_with_vocabulary_list
_members_cols = [
    'city', 'gender', 'registered_via'
]
for col in _members_cols:
    col_set = set(df_train[col].unique()).union(set(df_test[col].unique()))
    columns[col] = tf.feature_column.categorical_column_with_vocabulary_list(key=col, vocabulary_list=col_set)

for col in ['age', 'weird_age', 'validate_days',
                'registration_year', 'registration_month', 'registration_day',
                'expiration_year', 'expiration_month', 'expiration_day']:
    columns[col] = tf.feature_column.numeric_column(col)

# bucket the ages. 
# Bucketization allows us to find the music recommendtion by certain age groups
age_boundaries = [8, 18, 21, 24, 27, 30, 35, 40, 45, 50, 60]
columns['age_buckets'] = tf.feature_column.bucketized_column(
    columns['age'], boundaries=age_boundaries)

## 3.3 Song Related Feature

In [81]:
for col in ['artist_name', 'composer', 'lyricist', 'genre_ids']:
    col_set = set(df_train[col].unique()).union(set(df_test[col].unique()))
    print(col, len(col_set))

artist_name 46373
composer 86438
lyricist 37876
genre_ids 609


In [82]:
# Columns with a small set of categories: transform them using categorical_column_with_vocabulary_list
_songs_cols = ['language']
for col in _songs_cols:
    col_set = set(df_train[col].unique()).union(set(df_test[col].unique()))
    #print(col, len(col_set))
    columns[col] = tf.feature_column.categorical_column_with_vocabulary_list(key=col, vocabulary_list=col_set)

# artist_name, composer, lyricist, genre_ids
# Columns with a large set of categories: use hash bucket
columns['artist_name'] = tf.feature_column.categorical_column_with_hash_bucket('artist_name', hash_bucket_size=46400)
columns['composer'] = tf.feature_column.categorical_column_with_hash_bucket('composer', hash_bucket_size=86500)
columns['lyricist'] = tf.feature_column.categorical_column_with_hash_bucket('lyricist', hash_bucket_size=38000)
columns['genre_ids'] = tf.feature_column.categorical_column_with_hash_bucket('genre_ids', hash_bucket_size=610)

# numerical features
for col in ['song_length', 'song_year']:
    columns[col] = tf.feature_column.numeric_column(col)
# genre_count: number of genre 
# lyricist_count: number of lyricist
for col in ['genre_count', 'lyricist_count', 'composer_count', 'artist_count']:
    columns[col] = tf.feature_column.numeric_column(col)
# binary
for col in ['artist_composer', 'artist_composer_lyricist', 'is_featured', 'song_lang_boolean', 'smaller_song']:
    columns[col] = tf.feature_column.numeric_column(col)
# history
for col in ['count_song_played', 'count_artist_played']:
    columns[col] = tf.feature_column.numeric_column(col)

# song length (s) 
# bucketization allows us to find the music recommendtion by certain song length groups 
columns['song_length_buckets'] = tf.feature_column.bucketized_column(
    columns[col], boundaries=[x for x in [30, 60, 120, 180, 240, 300, 360, 420, 480, 540, 600, 800, 1200]])

##  3.4 Hash msno(usr_id) and song_id

In [83]:
for col in ['msno', 'song_id']:
    print(len(set(df_train[col]).union(set(df_test[col]))))

34403
419839


In [84]:
columns['msno'] = tf.feature_column.categorical_column_with_hash_bucket("msno", hash_bucket_size=34500)
columns['song_id'] = tf.feature_column.categorical_column_with_hash_bucket("song_id", hash_bucket_size=419839)

## 3.5 Final state

In [85]:
for key in columns.keys():
    print(key)

source_screen_name
source_type
source_system_tab
city
gender
registered_via
age
weird_age
validate_days
registration_year
registration_month
registration_day
expiration_year
expiration_month
expiration_day
age_buckets
language
artist_name
composer
lyricist
genre_ids
song_length
song_year
genre_count
lyricist_count
composer_count
artist_count
artist_composer
artist_composer_lyricist
is_featured
song_lang_boolean
smaller_song
count_song_played
count_artist_played
song_length_buckets
msno
song_id


In [86]:
# wide model input
# categorical feature
wide_columns_tag = [
    'song_id', 
    'msno', 
    'song_length_buckets', 
    'language', 
    'city',
    'source_screen_name',
    'source_type',
    'source_system_tab',
    'gender',
    'registered_via',
    'age_buckets',
    'artist_name',
    'composer',
    'lyricist',
    'genre_ids',
]
wide_columns = []
for tag in wide_columns_tag:
    wide_columns.append(columns[tag])

# cross column feature 
# these feature may be related so hash them together to produce new feature
wide_columns.append(tf.feature_column.crossed_column(['language', 'msno'], 
                                                     hash_bucket_size=int(1e4)))

wide_columns.append(tf.feature_column.crossed_column([columns['age_buckets'], 'artist_name'], 
                                                     hash_bucket_size=int(1e5)))

wide_columns.append(tf.feature_column.crossed_column(['artist_name', 'composer'], 
                                                     hash_bucket_size=int(1e6)))
wide_columns.append(tf.feature_column.crossed_column(['artist_name', 'composer', 'lyricist'], 
                                                     hash_bucket_size=int(1e6)))

In [87]:
# deep model input
# dense embedded feature
deep_columns = []
_categoricals = [
    'source_screen_name',
    'source_type',
    'source_system_tab',
    'language',
    'city',
    'registered_via'
]
# embed the categorical features for the deep part
for col in _categoricals:
    col_set = set(df_train[col].unique()).union(set(df_test[col].unique()))
    embedding_dimenstion = int(np.ceil(np.log2(len(col_set))))
    print(col, embedding_dimenstion)
    deep_columns.append(tf.feature_column.embedding_column(columns[col], dimension=embedding_dimenstion))
for col in ['msno', 'song_id']:
    deep_columns.append(tf.feature_column.embedding_column(columns[col], dimension=32))
for col in ['artist_name', 'composer', 'lyricist', 'genre_ids']:
    deep_columns.append(tf.feature_column.embedding_column(columns[col], dimension=16))
# gender only have three categories, so embedding is unnecessary
for col in ['gender']:
    deep_columns.append(tf.feature_column.indicator_column(columns[col]))
    
# numerical columns
_numericals = [
    'age', 
    'song_length',
    'genre_count', 
    'lyricist_count',
    'composer_count',
    'artist_count',
    'weird_age',
    'is_featured',
    'artist_composer',
    'artist_composer_lyricist',
    'song_lang_boolean',
    'smaller_song',
    'count_song_played',
    'count_artist_played',
    'registration_year',
    'registration_month',
    'registration_day',
    'expiration_year',
    'expiration_month',
    'expiration_day',
    'song_year',
]
for col in _numericals:
    deep_columns.append(columns[col])

source_screen_name 5
source_type 4
source_system_tab 4
language 4
city 5
registered_via 3


# 4 Modeling and evaluating

In [33]:
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2017-12-06 20:04:23


## 4.1 split into train and validation set

In [34]:
# split into train and validation set
from sklearn.model_selection import train_test_split
def split(X, y, val_size):
    '''
    split the data into training and validation set
    '''
    # test_num = int(val_size * X.shape[0])
    train_num = int((1 - val_size) * X.shape[0])
    return X[:train_num], X[train_num:], y[:train_num], y[train_num:]

In [35]:
X_train = df_train.drop(['target'], axis=1)
y_train = df_train['target']

X_test = df_test.drop(['id'], axis=1)
test_ids = df_test['id']

In [36]:
# X_trn, X_val, y_trn, y_val = split(X_train, y_train, val_size=0.1)
X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [37]:
print(X_trn.shape)
print(y_trn.shape)
print(X_val.shape)
print(y_val.shape)

(6639676, 35)
(6639676,)
(737742, 35)
(737742,)


## 4.2 Build model

In [88]:
# use DNNLinearCombinedClassifier estimator provided by tensorflow API
def build_estimator(model_dir):
    return tf.estimator.DNNLinearCombinedClassifier(
          model_dir=model_dir,
          linear_feature_columns=wide_columns,
          dnn_feature_columns=deep_columns,
          dnn_dropout=0.1,
          dnn_hidden_units=[128, 32]
    )

In [89]:
# input data to model (train: small batch, test: whole testdata)
batch_size = 128
print('batch_size:', batch_size)
def input_fn(X, Y, shuffle):
    return tf.estimator.inputs.pandas_input_fn(
            x=X,
            y=Y,
            batch_size=batch_size,
            num_epochs=1,
            shuffle=shuffle,
            num_threads=1,
            target_column='target'
        )

def test_input_fn():
    return tf.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=None,
        batch_size=batch_size,
        num_epochs=1,
        shuffle=False,
        num_threads=1
    )

batch_size: 128


In [90]:
# produce submission
result_dir = '../result/'
def produce(m, tag=''):
    print('Produce: ' + '='*30)
    print(time.strftime("%Y-%m-%d %H:%M:%S"))
    
    predictions = m.predict(input_fn=test_input_fn())
    predictions = list(predictions)
    predictions = list(map(lambda y: y['class_ids'][0], predictions))
    
    result_df = pd.DataFrame()
    result_df['id'] = test_ids
    result_df['target'] = predictions

    tag = str(tag)
    timestamp = time.strftime("%Y-%m-%d_%H_%M_%S")
    filename = result_dir + timestamp + '_' + tag + '.csv.gz'
    result_df.to_csv(filename, compression = 'gzip', index=False, float_format = '%.5f')
    print(time.strftime("%Y-%m-%d %H:%M:%S"))
    print('Produce Ends: ' + '='*30)

In [74]:
result_log = []
def print_result(result, tag):
    print('Result: ' + tag + '\t' + '-'*30)
    for key in sorted(result):
        to_print = "%s: %s" % (key, result[key])
        print(to_print)
    print('Result Ends: ' + tag + '\t' + '-'*30)

def evaluate(m, X, y, tag=''):
    global result_log
    print('Result:' + '='*30)
    print(time.strftime("%Y-%m-%d %H:%M:%S"))
    result = m.evaluate(input_fn=input_fn(X=X, Y=y, shuffle=False))
    print_result(result, tag)
    result_log.append((tag, result))
    print(time.strftime("%Y-%m-%d %H:%M:%S"))
    print('End:' + '='*30)

In [75]:
print(len(df_train.target))

7377418


# 5 Experiment

In [92]:
# build a model
model_dir = '../model/' + time.strftime("%Y_%m_%d_%H_%M_%S/")
print("model directory = %s" % model_dir)
m = build_estimator(model_dir)

model directory = ../model/2017_12_07_00_23_41/
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../model/2017_12_07_00_23_41/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000156A084DE80>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [93]:
# train, evaluate and produce output for Kaggle
print(time.strftime("%Y-%m-%d %H:%M:%S"))
for iter in range(21):
    m.train(input_fn=input_fn(X=X_trn, Y=y_trn, shuffle=True))
    if iter % 2 == 0:
        print(time.strftime("%Y-%m-%d %H:%M:%S"))
        # evaluate on training set
        evaluate(m, X_trn, y_trn, tag='train'+str(iter))
        # evaluate on validation set
        evaluate(m, X_val, y_val, tag='valid'+str(iter))
        # generate output for Kaggle submission
        produce(m, tag=iter)
print(time.strftime("%Y-%m-%d %H:%M:%S"))

2017-12-07 00:23:46
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ../model/2017_12_07_00_23_41/model.ckpt.
INFO:tensorflow:loss = 88.7229, step = 1
INFO:tensorflow:global_step/sec: 97.8743
INFO:tensorflow:loss = 85.024, step = 101 (1.024 sec)
INFO:tensorflow:global_step/sec: 144.332
INFO:tensorflow:loss = 83.8947, step = 201 (0.693 sec)
INFO:tensorflow:global_step/sec: 160.08
INFO:tensorflow:loss = 85.1228, step = 301 (0.625 sec)
INFO:tensorflow:global_step/sec: 160.609
INFO:tensorflow:loss = 84.833, step = 401 (0.623 sec)
INFO:tensorflow:global_step/sec: 164.306
INFO:tensorflow:loss = 82.0116, step = 501 (0.609 sec)
INFO:tensorflow:global_step/sec: 162.162
INFO:tensorflow:loss = 84.7779, step = 601 (0.617 sec)
INFO:tensorflow:global_step/sec: 162.44
INFO:tensorflow:loss = 83.0891, step = 701 (0.617 sec)
INFO:tensorflow:global_step/sec: 163.491
INFO:tensorflow:loss = 85.2912, step = 801 (0.611 sec)
INFO:tensorflow:global_step/sec: 169.913
INF

INFO:tensorflow:global_step/sec: 161.381
INFO:tensorflow:loss = 86.7364, step = 8401 (0.620 sec)
INFO:tensorflow:global_step/sec: 166.501
INFO:tensorflow:loss = 82.5046, step = 8501 (0.601 sec)
INFO:tensorflow:global_step/sec: 167.339
INFO:tensorflow:loss = 80.8809, step = 8601 (0.598 sec)
INFO:tensorflow:global_step/sec: 165.67
INFO:tensorflow:loss = 78.1098, step = 8701 (0.604 sec)
INFO:tensorflow:global_step/sec: 156.069
INFO:tensorflow:loss = 85.6743, step = 8801 (0.642 sec)
INFO:tensorflow:global_step/sec: 148.201
INFO:tensorflow:loss = 79.2384, step = 8901 (0.675 sec)
INFO:tensorflow:global_step/sec: 161.121
INFO:tensorflow:loss = 79.4547, step = 9001 (0.620 sec)
INFO:tensorflow:global_step/sec: 160.852
INFO:tensorflow:loss = 80.6679, step = 9101 (0.623 sec)
INFO:tensorflow:global_step/sec: 166.232
INFO:tensorflow:loss = 84.6904, step = 9201 (0.601 sec)
INFO:tensorflow:global_step/sec: 164.85
INFO:tensorflow:loss = 81.0106, step = 9301 (0.608 sec)
INFO:tensorflow:global_step/sec:

INFO:tensorflow:global_step/sec: 166.223
INFO:tensorflow:loss = 87.1101, step = 16801 (0.612 sec)
INFO:tensorflow:global_step/sec: 148.794
INFO:tensorflow:loss = 82.0349, step = 16901 (0.663 sec)
INFO:tensorflow:global_step/sec: 126.25
INFO:tensorflow:loss = 78.5036, step = 17001 (0.792 sec)
INFO:tensorflow:global_step/sec: 148.193
INFO:tensorflow:loss = 83.3617, step = 17101 (0.674 sec)
INFO:tensorflow:global_step/sec: 150.45
INFO:tensorflow:loss = 83.3555, step = 17201 (0.666 sec)
INFO:tensorflow:global_step/sec: 155.834
INFO:tensorflow:loss = 84.377, step = 17301 (0.641 sec)
INFO:tensorflow:global_step/sec: 161.381
INFO:tensorflow:loss = 78.4163, step = 17401 (0.620 sec)
INFO:tensorflow:global_step/sec: 151.802
INFO:tensorflow:loss = 77.9799, step = 17501 (0.660 sec)
INFO:tensorflow:global_step/sec: 151.112
INFO:tensorflow:loss = 82.0657, step = 17601 (0.662 sec)
INFO:tensorflow:global_step/sec: 147.754
INFO:tensorflow:loss = 78.4762, step = 17701 (0.677 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 154.386
INFO:tensorflow:loss = 76.9726, step = 25201 (0.648 sec)
INFO:tensorflow:global_step/sec: 160.603
INFO:tensorflow:loss = 78.7448, step = 25301 (0.622 sec)
INFO:tensorflow:global_step/sec: 153.436
INFO:tensorflow:loss = 83.5717, step = 25401 (0.653 sec)
INFO:tensorflow:global_step/sec: 148.413
INFO:tensorflow:loss = 76.6646, step = 25501 (0.675 sec)
INFO:tensorflow:global_step/sec: 142.478
INFO:tensorflow:loss = 80.671, step = 25601 (0.701 sec)
INFO:tensorflow:global_step/sec: 151.565
INFO:tensorflow:loss = 81.3029, step = 25701 (0.660 sec)
INFO:tensorflow:global_step/sec: 156.819
INFO:tensorflow:loss = 78.8747, step = 25801 (0.638 sec)
INFO:tensorflow:global_step/sec: 167.342
INFO:tensorflow:loss = 81.9513, step = 25901 (0.598 sec)
INFO:tensorflow:global_step/sec: 157.807
INFO:tensorflow:loss = 81.0128, step = 26001 (0.634 sec)
INFO:tensorflow:global_step/sec: 154.387
INFO:tensorflow:loss = 85.9848, step = 26101 (0.648 sec)
INFO:tensorflow:globa

INFO:tensorflow:global_step/sec: 157.558
INFO:tensorflow:loss = 81.2737, step = 33601 (0.634 sec)
INFO:tensorflow:global_step/sec: 153.201
INFO:tensorflow:loss = 79.6316, step = 33701 (0.653 sec)
INFO:tensorflow:global_step/sec: 134.226
INFO:tensorflow:loss = 79.0804, step = 33801 (0.746 sec)
INFO:tensorflow:global_step/sec: 143.508
INFO:tensorflow:loss = 85.587, step = 33901 (0.696 sec)
INFO:tensorflow:global_step/sec: 144.542
INFO:tensorflow:loss = 76.4287, step = 34001 (0.692 sec)
INFO:tensorflow:global_step/sec: 149.975
INFO:tensorflow:loss = 77.539, step = 34101 (0.667 sec)
INFO:tensorflow:global_step/sec: 152.733
INFO:tensorflow:loss = 79.5317, step = 34201 (0.655 sec)
INFO:tensorflow:global_step/sec: 156.814
INFO:tensorflow:loss = 77.5925, step = 34301 (0.638 sec)
INFO:tensorflow:global_step/sec: 157.309
INFO:tensorflow:loss = 76.7592, step = 34401 (0.636 sec)
INFO:tensorflow:global_step/sec: 148.857
INFO:tensorflow:loss = 88.3571, step = 34501 (0.673 sec)
INFO:tensorflow:global

...

INFO:tensorflow:global_step/sec: 165.95
INFO:tensorflow:loss = 74.7998, step = 131147 (0.604 sec)
INFO:tensorflow:global_step/sec: 157.059
INFO:tensorflow:loss = 78.6111, step = 131247 (0.637 sec)
INFO:tensorflow:global_step/sec: 146.455
INFO:tensorflow:loss = 74.6389, step = 131347 (0.682 sec)
INFO:tensorflow:global_step/sec: 160.344
INFO:tensorflow:loss = 77.4366, step = 131447 (0.624 sec)
INFO:tensorflow:global_step/sec: 157.884
INFO:tensorflow:loss = 75.597, step = 131547 (0.633 sec)
INFO:tensorflow:global_step/sec: 167.902
INFO:tensorflow:loss = 76.0268, step = 131647 (0.598 sec)
INFO:tensorflow:global_step/sec: 174.056
INFO:tensorflow:loss = 78.1826, step = 131747 (0.573 sec)
INFO:tensorflow:global_step/sec: 157.807
INFO:tensorflow:loss = 72.2552, step = 131847 (0.634 sec)
INFO:tensorflow:global_step/sec: 151.802
INFO:tensorflow:loss = 82.1151, step = 131947 (0.660 sec)
INFO:tensorflow:global_step/sec: 163.231
INFO:tensorflow:loss = 78.2332, step = 132047 (0.613 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 166.779
INFO:tensorflow:loss = 77.1273, step = 139447 (0.599 sec)
INFO:tensorflow:global_step/sec: 172.839
INFO:tensorflow:loss = 78.3318, step = 139547 (0.580 sec)
INFO:tensorflow:global_step/sec: 176.219
INFO:tensorflow:loss = 80.1131, step = 139647 (0.568 sec)
INFO:tensorflow:global_step/sec: 165.122
INFO:tensorflow:loss = 79.1882, step = 139747 (0.605 sec)
INFO:tensorflow:global_step/sec: 170.486
INFO:tensorflow:loss = 73.835, step = 139847 (0.587 sec)
INFO:tensorflow:global_step/sec: 171.364
INFO:tensorflow:loss = 73.6243, step = 139947 (0.584 sec)
INFO:tensorflow:global_step/sec: 159.065
INFO:tensorflow:loss = 74.9093, step = 140047 (0.629 sec)
INFO:tensorflow:global_step/sec: 162.433
INFO:tensorflow:loss = 73.7094, step = 140147 (0.616 sec)
INFO:tensorflow:global_step/sec: 171.068
INFO:tensorflow:loss = 73.1567, step = 140247 (0.585 sec)
INFO:tensorflow:global_step/sec: 168.471
INFO:tensorflow:loss = 80.3758, step = 140347 (0.594 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 168.755
INFO:tensorflow:loss = 78.0805, step = 147747 (0.593 sec)
INFO:tensorflow:global_step/sec: 171.364
INFO:tensorflow:loss = 81.2144, step = 147847 (0.584 sec)
INFO:tensorflow:global_step/sec: 167.902
INFO:tensorflow:loss = 76.1671, step = 147947 (0.596 sec)
INFO:tensorflow:global_step/sec: 164.578
INFO:tensorflow:loss = 72.1671, step = 148047 (0.608 sec)
INFO:tensorflow:global_step/sec: 171.356
INFO:tensorflow:loss = 80.075, step = 148147 (0.584 sec)
INFO:tensorflow:global_step/sec: 166.221
INFO:tensorflow:loss = 82.7659, step = 148247 (0.602 sec)
INFO:tensorflow:global_step/sec: 165.956
INFO:tensorflow:loss = 76.2159, step = 148347 (0.604 sec)
INFO:tensorflow:global_step/sec: 160.078
INFO:tensorflow:loss = 74.9923, step = 148447 (0.624 sec)
INFO:tensorflow:global_step/sec: 170.204
INFO:tensorflow:loss = 81.3429, step = 148547 (0.589 sec)
INFO:tensorflow:global_step/sec: 170.477
INFO:tensorflow:loss = 86.9934, step = 148647 (0.586 sec)
INFO:tensor

INFO:tensorflow:Restoring parameters from ../model/2017_12_07_00_23_41/model.ckpt-155619
INFO:tensorflow:Finished evaluation at 2017-12-07-05:54:12
INFO:tensorflow:Saving dict for global step 155619: accuracy = 0.660102, accuracy_baseline = 0.503408, auc = 0.724871, auc_precision_recall = 0.714869, average_loss = 0.615145, global_step = 155619, label/mean = 0.503408, loss = 78.7377, prediction/mean = 0.503019
Result: train2	------------------------------
accuracy: 0.660102
accuracy_baseline: 0.503408
auc: 0.724871
auc_precision_recall: 0.714869
average_loss: 0.615145
global_step: 155619
label/mean: 0.503408
loss: 78.7377
prediction/mean: 0.503019
Result Ends: train2	------------------------------
2017-12-07 00:54:12
End:==============================
Result:==============================
2017-12-07 00:54:12
INFO:tensorflow:Starting evaluation at 2017-12-07-05:54:15
INFO:tensorflow:Restoring parameters from ../model/2017_12_07_00_23_41/model.ckpt-155619
INFO:tensorflow:Finished evaluati

SystemError: <built-in function TF_NewBufferFromString> returned a result with an error set